Jusqu'à maintenant, nous avons vu avec MLflow la puissance du composant Tracking, qui permettait de fournir un historique des expérimentations réalisées en gardant une tracabilité des hyper-paramètres utilisés, des métriques obtenus mais également des artifacts générés (modèle, graphique).

Il y a également un autre composant particulièrement utile en tant que ML Engineer : c'est le **registre de modèle**. À l'instar d'un dépôt Git, le registre de modèle MLflow permettra de gérer plusieurs versions de modèles en leur attribuant également des tags, qui correspondent à des environnement de pré-production (*staging*) et de production.

<blockquote><p>🙋 <b>Ce que nous allons faire</b></p>
<ul>
    <li>Intégrer le tracking MLflow dans Kedro</li>
    <li>Créer un modèle packagé sous MLflow</li>
    <li>Versioner les modèles dans Kedro</li>
</ul>
</blockquote>

## Tracking MLflow sous Kedro

Dans le Notebook MLflow, nous avions entraîné un LightGBM manuellement et l'avons envoyé directement sur MLflow et Cloud Storage. L'objectif ici est de modifier le pipeline `training` sous Kedro pour y ajouter l'intégration avec MLflow.

Commençons par ajouter une variable d'environnement. Sous Python, le package `python-dotenv` est utile pour récupérer les variables d'environnement ou pour en configurer automatiquement lors du développement du projet.

À la racine du projet, créons le fichier `.env`.

Le rôle de ce fichier est identique à un `export` qui serait réalisé sur chaque variable. Le principal intérêt, en plus de pouvoir centraliser toutes les variables d'environnement, est que `python-dotenv` supporte aussi les variables d'environnement : il peut donc être utilisé à la fois lors du développement ou une fois le code en production.

À noter que nous aurions pu utiliser le fichier `parameters.yml` de Kedro. Dans la pratique, ce fichier est plutôt réservé aux paramètres des pipelines (ratio pour l'ensemble de test, hyper-paramètres par défaut, etc), et il est préférable, comme c'est le cas en développement logiciel, d'inscrire des paramètres plus généraux (comme les références aux servers, l'environnement de pré-production ou de production) dans des fichiers de configuration ou des variables d'environnement.

Créons le fichier `__init__.py` dans le dossier `training` des pipelines.

Pour rappel, ce fichier sera exécuté initialement par l'interpréteur Python, permettant ainsi de définir des configurations qui seront partagées par tous les fichiers du module/dossier. Par la suite, nous allons pouvoir appeler `os.getenv("MLFLOW_SERVER")` pour récupérer cette variable d'environnement dans n'importe quel fichier du dossier contenant ce `__init__.py`.

Nous allons ajouter deux paramètres dans `parameters.yml`.

Cela permet de décider à tout moment si l'on souhaite envoyer les logs et artifacts vers MLflow, ainsi que l'identifiant de l'expérience MLflow.

Dans `nodes.py` de `training`, modifiant la fonction `auto_ml` pour accepter ces deux nouveaux paramètres.

Par défaut, si aucune information n'est spécifiée concernant MLflow, on préfère ne pas envoyer de logs ou d'artifacts. Importons `os` ainsi que `mlflow`.

Au tout début de la fonction, après avoir récupéré la base d'apprentissage $(X, y)$, nous démarrons un run.

Si l'on souhaite tracker avec MLflow (`log_to_mlflow` à `True`), alors on spécifie l'URL de tracking, puis on lance un run avec `start_run` en indiquant l'identifiant de l'expérience associé au run.

À la fin de la fonction, nous envoyons les logs et artifacts vers MLflow. Elle retournera également le champ `mlflow_run_id`.

La dernière modification à apporter est de mettre à jour la définition du pipeline.

<img src="https://blent-learning-user-ressources.s3.eu-west-3.amazonaws.com/training/ml_engineer_facebook/img/model_registry3.png" />

Avant d'exécuter le pipeline, il faut penser à donner les droits d'écriture sur Cloud Storage au compte de service de Kedro. Rappelons-nous, nous avions crée un compte de service pour pouvoir récupérer les fichiers CSV sur le bucket. Pas besoin d'en créer un nouveau : nous pouvons utiliser celui existant en lui rajoutant de nouvelles autorisations.

Dirigeons-nous dans <a href="https://console.cloud.google.com/iam-admin/iam" target="_blank">IAM</a> et éditons le membre `purchase-predict@...`.

<img src="https://blent-learning-user-ressources.s3.eu-west-3.amazonaws.com/training/ml_engineer_facebook/img/model_registry1.png" />

Ajoutons le rôle Créateur des objets de l'espace de stockage. Après modification des rôles, il faut retélécharger une clé pour <a href="https://console.cloud.google.com/iam-admin/serviceaccounts" target="_blank">le compte de service</a>. Une fois le contenu de la clé modifié dans `conf/local/service-account.json`, nous devons mettre à jour la variable d'environnement `GOOGLE_APPLICATION_CREDENTIALS`.

À noter que pour aller plus vite, nous pouvons définir cette variable dans le fichier `.env`.

Temporairement, pour vérifier que notre code fonctionne, nous allons modifier le paramètre `automl_max_evals` à $1$. Dans le même temps, mettons en place le pipeline `global` qui va exécuter de manière séquentielle les trois pipelines `loading`, `processing` et `traning`.

Exécutons le pipeline `global`.

Après quelques dizaines de secondes, le run devrait être visible sous MLflow.

<img src="https://blent-learning-user-ressources.s3.eu-west-3.amazonaws.com/training/ml_engineer_facebook/img/model_registry2.png" />

N'oublions pas, puisque le test est concluant, de remettre `automl_max_evals` à sa valeur d'origine.

## Pipeline de déploiement vers MLflow

Pour l'instant, nous avons uniquement effectué du tracking. Bien que cela soit pratique pour historiser et garder une trace des différentes exécutions des pipelines d'entraînement, cela ne permet pas directement de gérer efficacement des **versions de modèles**.

### Registre de modèles

Le **Model Registry** (registre de modèles) est un composant de MLflow qui permet de gérer des versions de modèles de Machine Learning, en proposant également des **stages** (états).

- Le tag **staging** correspond à un modèle considéré comme pré-production.
- Le tag **production** correspond à un modèle qui serait en environnement de production.
- Le tag **archived** pour les anciens modèles staging ou production archivés.

C'est un composant particulièrement utile pour gérer le cycle de vie des modèles, car le cycle staging, production et archive est couramment appliqué lorsque des modèles sont mis à jour régulièrement. Sous MLflow, l'onglet Models permet d'afficher tous les modèles enregistrés.

<img src="https://blent-learning-user-ressources.s3.eu-west-3.amazonaws.com/training/ml_engineer_facebook/img/model_registry4.png" />

Retournons dans les expériences et choisissons le dernier run que nous avons lancé. En cliquant sur `model` dans les artifacts, un bouton *Register Model* apparaît à droite.

<img src="https://blent-learning-user-ressources.s3.eu-west-3.amazonaws.com/training/ml_engineer_facebook/img/model_registry5.png" />

En cliquant dessus, nous allons pouvoir ajouter manuellement le modèle au registre. Pour cela, nous devons créer un nouveau modèle que l'on nommera `purchase_predict`.

<img src="https://blent-learning-user-ressources.s3.eu-west-3.amazonaws.com/training/ml_engineer_facebook/img/model_registry6.png" />

En retournant dans Models, nous voyons que la version 1 (le modèle que nous venons d'ajouter est bien présent).

<img src="https://blent-learning-user-ressources.s3.eu-west-3.amazonaws.com/training/ml_engineer_facebook/img/model_registry7.png" />

En cliquant dessus, nous avons accès à plus de détails sur les différentes versions présentes.

<img src="https://blent-learning-user-ressources.s3.eu-west-3.amazonaws.com/training/ml_engineer_facebook/img/model_registry8.png" />

Pour une version spécifique, nous pouvons manuellement transitionner vers un état de pré-production ou de production. L'objectif sera d'automatiser cette tâche pour automatiquement changer l'état d'un modèle que l'on aura entraîné.

<img src="https://blent-learning-user-ressources.s3.eu-west-3.amazonaws.com/training/ml_engineer_facebook/img/model_registry9.png" />

### Pipeline de déploiement

Une fois l'entraînement réalisé, nous allons également implémenter dans Kedro un pipeline qui va permettre de transitionner l'état d'un modèle en staging ou production. Pour cela, nous allons créer un pipepline `deployment` avec les trois fichiers `__init__.py`, `nodes.py` et `pipeline.py`. Le fichier `__init__.py` contiendra les mêmes instructions.

Nous allons créer deux fonctions dans le fichier `nodes.py`.

- La fonction `push_to_model_registry` va envoyer un modèle enregistré dans tracking vers le registre de modèle associé. Notons que pour cela, le modèle doit déjà être dans le tracking.
- La fonction `stage_model` qui permet de transitionner un modèle du registre vers un état staging ou production.

Nous avons besoin de configurations supplémentaires, `registry_name`, pour référencer le nom du modèle dans le registre, que l'on ajoute au fichier `parameters.yml`, ainsi que la variable d'environnement `ENV` pour spécifier si le projet Kedro est exécuté dans un environnement de pré-production ou de production. Modifions le fichier `.env`.

De même pour le fichier `parameters.yml`.

Le paramètre `run_id` sera lui fourni par le catalogue de données. Écrivons le code du pipeline, qui ne présente aucune difficulté.

Il ne reste plus qu'à renseigner ce pipeline dans `hooks.py`.

Nous obtenons le pipeline suivant.

<img src="https://blent-learning-user-ressources.s3.eu-west-3.amazonaws.com/training/ml_engineer_facebook/img/model_registry10.png" />

Lançons une exécution globale (en spécifiant là-aussi le paramètre `automl_max_evals` à 1 pour accélerer les calculs.

Puisque nous sommes en environnement staging, sur MLflow, le modèle enregistré doit avoir l'état correspondant.

<img src="https://blent-learning-user-ressources.s3.eu-west-3.amazonaws.com/training/ml_engineer_facebook/img/model_registry12.png" />

Nous avons donc à la fois la *Latest Version* à 2, et c'est elle qui est actuellement en staging.

L'intégralité des pipelines ont été réalisées, ce qui donne pour le pipeline `global` un ensemble assez important d'étapes.

<img src="https://blent-learning-user-ressources.s3.eu-west-3.amazonaws.com/training/ml_engineer_facebook/img/model_registry11.png" />

> ❓ En quoi envoyer le modèle vers le registre peut nous être utile ?

Ce qui va être puissant, c'est que l'on sera capable de récupérer, dans un projet différent que Kedro, le modèle le plus à jour. Ainsi, on **découple fortement** la phase d'expérimentation/d'entraînement du modèle et la phase de déploiement, qui est une bonne pratique du Cloud.

Comme toujours, il ne faut pas hésiter à régulièrement pousser son code vers le dépôt Git.

Nous allons créer une **nouvelle branche** appelée staging. Cela va nous permettre de produire un code en environnement de pré-production.

On n'oubliera pas d'ajouter la clé SSH à l'agent avec de pousser vers le dépôt.

Sur <a href="https://source.cloud.google.com/" target="_blank">Cloud Source</a>, la branche `staging` est désormais visible.

## ✔️ Conclusion

Dorénavant, le projet Kedro peut exécuter le pipeline ML de A à Z avec MLflow pour centraliser les différents modèles qui seront entraînés.

- Nous avons intégré le tracking MLflow sous Kedro.
- Nous avons créer un modèle packagé sous MLflow.
- Nous sommes capable de changer les états des modèles de pré-production et de production automatiquement.

> ➡️ Nous avons un modèle prêt à être utilisé. Mais pour être utilisé, il faut le rendre accessible ... et la meilleure manière de le faire, c'est de <b>construire une API</b>.